In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from pathlib import Path
from kspecdr.tlm.make_tlm import make_tlm

WD = Path.cwd()
datadir = Path("/data1/hbahk/kspec/kspecdr/resources/")
# fpath = datadir / "08feb10087im.fits"
fpath = datadir / "08feb10093.fits"
args = {"IMAGE_FILENAME": fpath.as_posix()}

In [41]:
from kspecdr.io.image import ImageFile

im = ImageFile(fpath.as_posix(), mode='READ')
im.open()
print(im.get_header_value('RO_NOISE', 7.0))
print(im.get_header_value('RO_GAIN'))

INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits (mode: READ)
6.87
1.87


In [46]:
from kspecdr.preproc.make_im import make_im

make_im(fpath.as_posix(),
        cosmic_ray_method='NONE',
        verbose=True)

INFO:kspecdr.preproc.make_im:==================================================
INFO:kspecdr.preproc.make_im:Preprocessing image data contained in RAW frame
INFO:kspecdr.preproc.make_im:==================================================
INFO:kspecdr.preproc.make_im:RAW file = /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits
INFO:kspecdr.preproc.make_im:Creating IM file from raw data with TDFIO_CREATEBYCOPY functionality...
INFO:kspecdr.preproc.make_im:Creating IM file from raw file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits -> /data1/hbahk/kspec/kspecdr/resources/08feb10093_im.fits


INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits (mode: READ)
INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits (mode: READ)
INFO:kspecdr.preproc.make_im:Instrument: TAIPAN, SPECTID: RD, CLASS: , BITPIX: 16
INFO:kspecdr.preproc.make_im:Processing TAIPAN raw data: transposing axes
INFO:kspecdr.io.image:Closed file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits
INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits (mode: READ)
INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits (mode: READ)
INFO:kspecdr.preproc.make_im:Creating variance HDU for raw file
INFO:kspecdr.preproc.make_im:No fiber table found in source file
INFO:kspecdr.io.image:Closed file: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits
INFO:kspecdr.preproc.make_im:Successfully created IM file: /data1/hbahk/kspec/kspecdr/resources/08feb10093_im.fits
INFO:kspecdr.io.imag

'/data1/hbahk/kspec/kspecdr/resources/08feb10093_im.fits'

In [30]:
make_tlm(args)

INFO:kspecdr.tlm.make_tlm:Generating tramline map from /data1/hbahk/kspec/kspecdr/resources/08feb10087im.fits
INFO:kspecdr.tlm.make_tlm:Instrument code: 6
INFO:kspecdr.tlm.make_tlm:Starting tramline map generation for non-2DF instrument


INFO:kspecdr.tlm.make_tlm:Fibres officially in use: 0
INFO:kspecdr.tlm.make_tlm:Fibres potentially able: 1000
INFO:kspecdr.tlm.make_tlm:Fibres officially dead: 0
INFO:kspecdr.tlm.make_tlm:Detecting traces in image data


NotImplementedError: Trace detection algorithm not yet implemented. This should implement the LOCATE_TRACES functionality from the Fortran code.

In [35]:
from astropy.io import fits

hdul = fits.open(fpath)
hdul.info()

Filename: /data1/hbahk/kspec/kspecdr/resources/08feb10093.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      98   (2048, 2098)   int16 (rescales to uint16)   
  1  STRUCT.MORE.FIBRES    1 BinTableHDU     96   300R x 23C   [80A, 1D, 1D, 1D, 1D, 1D, 1D, 1J, 1J, 1D, 1I, 1I, 1A, 1J, 1J, 1I, 1I, 1I, 1D, 80A, 1D, 1D, 20A]   


In [32]:
from astropy.table import Table
Table(hdul[-1].data)

NAME
str10
MFARC


In [22]:
"FIBRES" in hdul[-1].name

True

In [36]:
header = hdul[0].header
header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2048 / length of data axis 1                          
NAXIS2  =                 2098 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BZERO   =                32768 / offset data range to that of unsigned short    
BSCALE  =                    1 / default scaling factor                         
LONGSTRN= 'OGIP 1.0'           / The HEASARC Long String Convention may be used.
COMMENT   This FITS file may contain long string keyword values that are        
COMMENT   continued over mul

In [28]:
hdul.index_of("PRIMARY")

0

In [37]:
hdul[hdul.index_of("PRIMARY")].writeto("test.fits")

In [38]:
thdul = fits.open("test.fits")

# You Should HANDLE THE ZERO VARIANCE!